In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from urllib.request import urlopen
from datetime import datetime
from tqdm import tqdm
import time

In [2]:
#harakah daily article from 2017 - todays date 21.5.2023
start_date = '2017/01/01'
today = datetime.now()
dateRange = pd.date_range(start_date,today).strftime("%Y/%m/%d")

In [6]:
hrefs = []
for i in tqdm(dateRange):

    x = 1

    while True:

        while True:
            try:
                r = requests.get(f'https://harakahdaily.net/index.php/{i}/page/{x}')
                break
            except:
                time.sleep(1.0)
        

        soup = BeautifulSoup(r.text, "lxml")

        div = soup.find_all('h1', attrs = {"class":"entry-title td-page-title"})

        if len(div) == 0:
            break

        for link in soup.find_all('div', attrs = {"class":"td_module_10 td_module_wrap td-animation-stack"}):
            
            category = link.find('a', class_='td-post-category')

            if category and category.text.strip().lower() != 'english':
                href = link.find('a', class_='td-image-wrap').get('href')
                hrefs.append(href)
                    
        
        x = x + 1




  2%|▏         | 35/2333 [00:15<12:20,  3.10it/s]

In [6]:
with open('article-links-harakah.json') as fopen:
    hrefs = json.load(fopen)

In [7]:
len(hrefs)

45667

In [8]:
import requests
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
import json

data = []
batch_size = 64

for x in tqdm(hrefs):

    while True:
        try:

            r = requests.get(x)
            
            break

        except Exception as e:
            print(e)
            time.sleep(5.0)

        
    soup = BeautifulSoup(r.text, "lxml")

    try:
        h = soup.find('h1', class_="entry-title")
        title = h.text
    except Exception as e:
        print(e)
        continue



    p = [p.text for p in soup.find_all("p") if len(p.text.split()) > 20]

    if len(p) == 0:
            h = soup.find('div', class_="td-post-content tagdiv-type")
            p = [p.text for p in h.find_all('div') if len(p.text.split()) > 20 and p.get('class') == None]


    data = {'url': x, 'headline': title, 'content': p}



    # Write the batch to the JSON file
    with open('article-harakah.jsonl', 'a') as f:
        json.dump(data, f)
        f.write('\n')




 11%|█         | 4910/45667 [34:53<4:12:36,  2.69it/s] 

'NoneType' object has no attribute 'text'


 11%|█         | 5076/45667 [36:11<4:33:05,  2.48it/s] 

HTTPSConnectionPool(host='harakahdaily.net', port=443): Max retries exceeded with url: /index.php/2018/03/13/roboh-pasar-pasir-puteh-tidak-patuhi-undang-undang/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001FC012E41D0>, 'Connection to harakahdaily.net timed out. (connect timeout=None)'))


 11%|█▏        | 5205/45667 [37:31<4:45:01,  2.37it/s] 

...

HTTPSConnectionPool(host='harakahdaily.net', port=443): Max retries exceeded with url: /index.php/2023/03/02/laksanakan-model-gaji-progresif-di-malaysia/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001FC01DD0810>, 'Connection to harakahdaily.net timed out. (connect timeout=None)'))


 97%|█████████▋| 44321/45667 [6:09:30<09:42,  2.31it/s]  

HTTPSConnectionPool(host='harakahdaily.net', port=443): Max retries exceeded with url: /index.php/2023/03/22/tun-m-bidas-ph-dalam-tiga-isu/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001FC01046550>, 'Connection to harakahdaily.net timed out. (connect timeout=None)'))


100%|██████████| 45667/45667 [6:21:07<00:00,  2.00it/s]  


: 

In [1]:
import langid

In [14]:
import jsonlines
import ast
import codecs

# Specify the input and output file paths
input_file = "D:/Aisyah/eleuther-ai ASEAN/harakah daily/article-harakah.jsonl"
output_file = "D:/Aisyah/eleuther-ai ASEAN/harakah daily/article-harakah-cleaned.jsonl"

langid.set_languages(['ms', 'en'])  #ISO 639-1 codes

# Open the input and output files
with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
    # Iterate over each line in the input file
    for line in reader:
        # Check if the content field is empty ([] or {})
        if line['content']:
            # Write the non-empty line to the output file
            decoded_list = []
            for string in line['content']:
                try:
                    decoded_string = codecs.escape_decode(string)[0].decode('utf-8')
                except ValueError:
                    decoded_string = string
                decoded_list.append(decoded_string)
            line['content'] = decoded_list

            writer.write(line)

          

C:\Users\User\AppData\Local\Temp\ipykernel_3476\1062229075.py:23: DeprecationWarning: invalid escape sequence '\I'
  decoded_string = codecs.escape_decode(string)[0].decode('utf-8')
C:\Users\User\AppData\Local\Temp\ipykernel_3476\1062229075.py:23: DeprecationWarning: invalid escape sequence '\?'
  decoded_string = codecs.escape_decode(string)[0].decode('utf-8')
C:\Users\User\AppData\Local\Temp\ipykernel_3476\1062229075.py:23: DeprecationWarning: invalid escape sequence '\Y'
  decoded_string = codecs.escape_decode(string)[0].decode('utf-8')
C:\Users\User\AppData\Local\Temp\ipykernel_3476\1062229075.py:23: DeprecationWarning: invalid escape sequence '\â'
  decoded_string = codecs.escape_decode(string)[0].decode('utf-8')


: 